In [ ]:
# Math Section for Skuld model v2
import math
import numpy as np

a = 1 # the main part of what i will be changing for weights changes the size of the sigmoid without changing near zero
b = 0  # bias will add to updating with ^ later
m = 0.5 # the rate of change of the sigmoid 
c = 1 # a constant I used, same as but but shift the sigmod to a place where i 0 is close to 0 (actuyll 0.04 but whatever)
layer_bias = -0.5


def ians_sigmoid(x, a_a, b_b, m_m, c_c, layer=0):
    # width = max(width, 0.01)  # prevent division by 0 or instability
    # layer_shift = layer * 0.1  # optional small bias per layer
    # return scale * np.exp(-((x - center - layer_shift) ** 2) / (2 * width ** 2))
    return (1/ ((1/a_a) + np.exp(((-x*m_m) + b_b + c_c + layer * layer_bias))))* a_a
    #return 1/ ((1/a_a) + np.exp(((-x*m_m) + b_b + c_c + layer * layer_bias)))
#print(ians_sigmoid(0.8, a, b, m, c))

# def ians_sigmoid_new(x, a_a, b_b, m_m, c_c, layer=0):
#     return (1/ ((1/a_a) + np.exp(((-x*m_m) + b_b + c_c + layer * layer_bias))))* a_a

def calc_loss(output, desired):
    # can change but not a requirement yet
    return (0.5 * ((output - desired) ** 2))


def d_loss_d_weight_2(n1, n2, desired):
    # check the removal of the layer bias fo rthis one and go back to check the others as well
    return ((n2 - desired) * (n2) * (1-(n2)) * n1) # removedall layer_bias can check layer if correct call

def d_loss_d_weight_1(n1, n2, desired, w2, input):
    # check the removal of the layer bias fo rthis one and go back to check the others as well
    return ((n2 - desired) * (n2) * (1-(n2)) * n1 * w2 * (1- n1) * input)


def d_loss_d_bias_2(n2, desired):
    return (n2 - desired) * (n2 - layer_bias) * (1 - (n2 - layer_bias))

def d_loss_d_bias_1(n1, n2, delta2, desired, a2):
    #delta2 = (n2 - desired) * (n2 - layer_bias) * (1 - (n2 - layer_bias))
    return delta2 * a2 * (n1 - layer_bias) * (1 - (n1 - layer_bias))


def update_param(lr, grad, w):
    w -= lr * grad
    return w


Output after one update: [0.37626079]


In [63]:

def forward_pass(inputs, a, b):
    global m, c
    sig_r1 = ians_sigmoid(inputs[0], a[0], b[0], m, c)
    delay_1 = sig_r1 + inputs[0]
    sig_r2 = ians_sigmoid(inputs[1], a[1], b[1], m, c)
    delay_2 = sig_r2 + inputs[1]
    avg_delay = (delay_1 + delay_2) / 2

    sig_r2 = ians_sigmoid(avg_delay, a[2], b[2], m, c, layer=1)
    delay_3 = sig_r2 + avg_delay

    return ([delay_1, delay_2, delay_3])


def run_basic(input, a, b, desired, lr):   # will need to check work on bias/b and make sure everything is correct in this

    result = forward_pass(inputs, a, b)
   
    n1 = result[0]
    n2 = result[1]
    n3 = result[2]
    print(n1, n2, n3)

    #loss = calc_loss(output, desired)

    # delta_a2 = - d_loss_d_weight_2(n1, n2, desired)

    # delta_a1 = - d_loss_d_weight_1(n1, n2, desired, a2, input) # trying to just input a as a the weight directly

    # new_a1 = update_param(lr, delta_a1, a1)
    # new_a2 = update_param(lr, delta_a2, a2)


    # delta_b2 = d_loss_d_bias_2(n2, desired)
    # delta_b1 = d_loss_d_bias_1(n1, n2, delta_b2, desired, b2) # trying to just input a as a the weight directly


    # new_b1 = update_param(lr, delta_b1, b1)
    # new_b2 = update_param(lr, delta_b2, b2)

    # return new_a1, new_a2, new_b1, new_b2, result
    return

m = 0.5
c = 1 
inputs = [0.5, 0.8]
a = [1.0]*3
b = [0.0]*3
desired = 1.8

run_basic(inputs, a, b, desired, 0.1)

0.820821300824607 1.1543436937742046 1.4860303144480338


In [71]:
s1 = ians_sigmoid(inputs[0], 1, 0, 0.5, 1, layer=0)  + inputs[0]

s2 = ians_sigmoid(inputs[1], 1, 0, 0.5, 1, layer=0) + inputs[1]

s3 = ians_sigmoid((s1 + s2)/2, 1, 0, 0.5, 1, layer=1) + (s1 + s2)/2

print(s1, s2, s3)

0.820821300824607 1.1543436937742046 1.4860303144480338


In [4]:
import numpy as np

# 1. Standard sigmoid + derivative
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def sigmoid_deriv(z):
    s = sigmoid(z)
    return s * (1 - s)

class NeuralNetwork:
    def __init__(self, input_size, hidden_size, output_size):
        # Initialize weights and biases
        self.W1 = np.random.randn(hidden_size, input_size)
        self.b1 = np.zeros((hidden_size, 1))
        self.W2 = np.random.randn(output_size, hidden_size)
        self.b2 = np.zeros((output_size, 1))

    def forward(self, x):
        # Hidden layer
        self.z1 = self.W1 @ x + self.b1
        self.a1 = sigmoid(self.z1)
        # Output layer
        self.z2 = self.W2 @ self.a1 + self.b2
        self.a2 = sigmoid(self.z2)
        return self.a2

    def compute_loss(self, y_hat, y):
        return 0.5 * np.sum((y_hat - y) ** 2)

    def backward(self, x, y):
        # 1) Output error
        delta2 = (self.a2 - y) * sigmoid_deriv(self.z2)
        # 2) Hidden error
        delta1 = (self.W2.T @ delta2) * sigmoid_deriv(self.z1)

        # Parameter gradients
        self.dW2 = delta2 @ self.a1.T
        self.db2 = delta2
        self.dW1 = delta1 @ x.T
        self.db1 = delta1

    def update_params(self, lr):
        # Gradient descent step
        self.W2 -= lr * self.dW2
        self.b2 -= lr * self.db2
        self.W1 -= lr * self.dW1
        self.b1 -= lr * self.db1

    def train_step(self, x, y, lr=0.1):
        y_hat = self.forward(x)
        loss = self.compute_loss(y_hat, y)
        self.backward(x, y)
        self.update_params(lr)
        return loss, y_hat

# Example: X=3 inputs, Y=4 hidden, Z=2 outputs
np.random.seed(42)
model = NeuralNetwork(input_size=3, hidden_size=4, output_size=2)

# Dummy data (column vectors)
x = np.array([[0.8], [0.1], [0.3]])
y = np.array([[1.0], [1.5]])

# Training loop
n_epochs = 10
lr = 0.1

for epoch in range(1, n_epochs + 1):
    loss, y_hat = model.train_step(x, y, lr)
    # Flatten prediction for readability
    prediction = y_hat.flatten()
    print(f"Epoch {epoch:2d} — Loss: {loss:.5f} — Predicted: {prediction}")


Epoch  1 — Loss: 1.38932 — Predicted: [0.05066474 0.12981888]
Epoch  2 — Loss: 1.38057 — Predicted: [0.05147972 0.13564856]
Epoch  3 — Loss: 1.37130 — Predicted: [0.05231687 0.14187985]
Epoch  4 — Loss: 1.36145 — Predicted: [0.05317686 0.14854387]
Epoch  5 — Loss: 1.35101 — Predicted: [0.05406037 0.15567347]
Epoch  6 — Loss: 1.33992 — Predicted: [0.05496807 0.16330303]
Epoch  7 — Loss: 1.32816 — Predicted: [0.05590063 0.17146811]
Epoch  8 — Loss: 1.31569 — Predicted: [0.05685871 0.18020503]
Epoch  9 — Loss: 1.30247 — Predicted: [0.05784296 0.18955019]
Epoch 10 — Loss: 1.28848 — Predicted: [0.05885402 0.19953927]


In [5]:
import numpy as np

# 1. Standard sigmoid + derivative
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def sigmoid_deriv(z):
    s = sigmoid(z)
    return s * (1 - s)

class SNN_Network:
    def __init__(self, input_size, hidden_size, output_size):
        # Initialize weights and biases
        self.W1 = np.random.randn(hidden_size, input_size)
        self.b1 = np.zeros((hidden_size, 1))
        self.W2 = np.random.randn(output_size, hidden_size)
        self.b2 = np.zeros((output_size, 1))

    def forward(self, x):
        # Sum of raw inputs (scalar)
        sum_in = np.sum(x)
        # Hidden layer pre-activation
        self.z1 = self.W1 @ x + self.b1
        # Hidden activation ("spike response")
        self.a1 = sigmoid(self.z1)
        # Delay accumulator: add summed raw input to each hidden neuron
        self.d1 = self.a1 + sum_in
        
        # Sum of delayed-hidden (scalar)
        sum_h = np.sum(self.d1)
        # Output layer pre-activation
        self.z2 = self.W2 @ self.d1 + self.b2
        # Output activation
        self.a2 = sigmoid(self.z2)
        # Delay accumulator for output
        self.d2 = self.a2 + sum_h
        
        return self.d2  # use the delayed output as the final "guess"

    def compute_loss(self, y_pred, y):
        return 0.5 * np.sum((y_pred - y) ** 2)

    def backward(self, x, y):
        # Backprop through the final sigmoid
        delta2 = (self.a2 - y) * sigmoid_deriv(self.z2)
        # Gradients for W2 and b2
        self.dW2 = delta2 @ self.d1.T
        self.db2 = delta2
        
        # Backprop into hidden (through W2 and sigmoid)
        delta1 = (self.W2.T @ delta2) * sigmoid_deriv(self.z1)
        # Gradients for W1 and b1
        self.dW1 = delta1 @ x.T
        self.db1 = delta1

    def update(self, lr):
        self.W2 -= lr * self.dW2
        self.b2 -= lr * self.db2
        self.W1 -= lr * self.dW1
        self.b1 -= lr * self.db1

    def train_step(self, x, y, lr=0.1):
        y_pred = self.forward(x)
        loss = self.compute_loss(y_pred, y)
        self.backward(x, y)
        self.update(lr)
        return loss, y_pred

# Example: X=3 inputs, Y=4 hidden, Z=2 outputs, summing inputs at each layer
np.random.seed(42)
model = SNN_Network(input_size=3, hidden_size=4, output_size=2)

# Dummy data (column vectors)
x = np.array([[0.8], [0.1], [0.3]])
y = np.array([[1.0], [0.0]])

# Training loop
n_epochs = 10
lr = 0.1

for epoch in range(1, n_epochs + 1):
    loss, y_pred = model.train_step(x, y, lr)
    print(f"Epoch {epoch:2d} — Loss: {loss:.5f} — Predicted (delayed): {y_pred.flatten()}")


Epoch  1 — Loss: 49.64083 — Predicted (delayed): [7.52598245 7.52949007]
Epoch  2 — Loss: 49.64068 — Predicted (delayed): [7.52597187 7.52947909]
Epoch  3 — Loss: 49.64053 — Predicted (delayed): [7.52596128 7.52946809]
Epoch  4 — Loss: 49.64038 — Predicted (delayed): [7.52595069 7.52945709]
Epoch  5 — Loss: 49.64023 — Predicted (delayed): [7.52594009 7.52944608]
Epoch  6 — Loss: 49.64007 — Predicted (delayed): [7.52592948 7.52943506]
Epoch  7 — Loss: 49.63992 — Predicted (delayed): [7.52591887 7.52942404]
Epoch  8 — Loss: 49.63977 — Predicted (delayed): [7.52590824 7.529413  ]
Epoch  9 — Loss: 49.63962 — Predicted (delayed): [7.52589762 7.52940197]
Epoch 10 — Loss: 49.63946 — Predicted (delayed): [7.52588698 7.52939092]
